In [39]:
import os, numpy, torch
from torch.utils.data import DataLoader

from matplotlib import pyplot as plt
from matplotlib import colors, colormaps
#brain visualization import
from nilearn import regions, datasets, surface, plotting, image, maskers
from nilearn.plotting import plot_roi, plot_stat_map
#ridge regression
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import GroupKFold

#from library
import sys
sys.path.append('/home/maellef/git/cNeuromod_encoding_2020')
from models import encoding_models as encod
import files_utils as fu
from Datasets_utils import SequentialDataset, create_usable_audiofmri_datasets

In [2]:
MIST_path = '/home/maellef/DataBase/fMRI_parcellations/MIST_parcellation/Parcellations/MIST_ROI.nii.gz'
voxel_mask = '/home/maellef/git/cNeuromod_encoding_2020/parcellation/STG_middle.nii.gz'
dir_path = "/home/maellef/Results/Phantom_general"

In [3]:
def voxels_nii(voxel_data, voxel_mask, t_r=1.49):
#from voxels to nii
    voxel_masker = maskers.NiftiMasker(mask_img=voxel_mask, standardize=False, 
                                       detrend=False, t_r=t_r, smoothing_fwhm=8)
    voxel_masker.fit()
    vox_data_nii = voxel_masker.inverse_transform(voxel_data)
    return vox_data_nii

In [4]:
def surface_fig(parcel_data, vmax, threshold=0, cmap='turbo', inflate=True, colorbar=True, no_background=True):     
    nii_data = regions.signals_to_img_labels(parcel_data, MIST_path)
    fig, ax = plotting.plot_img_on_surf(nii_data,
                              views=['lateral', 'medial'], hemispheres=['left', 'right'], inflate=inflate,
                              vmax=vmax, threshold=threshold, colorbar=colorbar, cmap=cmap, symmetric_cbar=False)
    return fig

In [5]:
def voxel_map(voxel_data, vmax=None, cut_coords=None, tr = 1.49, bg_img=None, cmap = 'cold_hot') : 
    f = plt.Figure()
    data_nii = voxels_nii(voxel_data, voxel_mask, t_r=tr)
    if bg_img is not None : 
        plotting.plot_stat_map(data_nii, bg_img=bg_img, draw_cross=False, vmax=vmax,
                           display_mode='x', cut_coords=[-63, -57, 57, 63], figure=f,
                              black_bg=True, dim = 0, cmap=cmap)
    else :
        plotting.plot_stat_map(data_nii, draw_cross=False, vmax=vmax,
                           display_mode='x', cut_coords=[-63, -57, 57, 63], figure=f)
    return f

In [6]:
def extend_colormap(original_colormap = 'twilight', percent_start = 0.25, percent_finish = 0.25):
    colormap = colormaps[original_colormap]
    nb_colors = colormap.N
    new_colors_range = colormap(numpy.linspace(0,1,nb_colors))

    n_start = round(nb_colors/(1-percent_start)) - nb_colors if percent_start != 0 else 0
    new_color_start = numpy.array([colormap(0)]*n_start).reshape(-1, new_colors_range.shape[1])
    n_finish = round(nb_colors/(1-percent_finish)) - nb_colors if percent_finish != 0 else 0
    new_color_finish = numpy.array([colormap(0)]*n_finish).reshape(-1, new_colors_range.shape[1])

    new_colors_range = numpy.concatenate((new_color_start,new_colors_range,new_color_finish), axis=0)
    new_colormap = colors.ListedColormap(new_colors_range)
    return new_colormap

In [ ]:
for dataset in ['friends', 'movie10']:
    subdatasets = ['s04'] if dataset == 'friends' else ['bourne', 'figures', 'life', 'wolf']
    for sub_ds in subdatasets:
        dspath = os.path.join(dir_path, dataset, sub_ds)
        for file in os.listdir(dspath):
            sub = file[:6]
            conv = 'conv4' if 'conv4' in file else 'baseline'
            scale = 'WB' if 'MIST_ROI' in file else 'STG'
            
            filepath = os.path.join(dspath, file)
            arr = numpy.load(filepath)
            print(arr.shape)

            parcel_data = numpy.mean(arr, axis=0).reshape(1, -1)
            vmax = numpy.max(numpy.abs(parcel_data))
            print(dataset, sub_ds, sub, scale, conv)
            print(f'min :', numpy.min(parcel_data), f', max : ', numpy.max(parcel_data), f', absolute max : ', vmax)

            #cmp = extend_colormap(original_colormap = 'turbo', percent_start=0.5, percent_finish=0) 
            if scale == 'WB' :
                fig = surface_fig(parcel_data, vmax=None, threshold = 0.1)
            else :
                fig = voxel_map(parcel_data)
        
            output_file = os.path.join(dir_path, 'maps', '{}_{}_{}_{}_{}'.format(dataset, sub_ds, sub, scale, conv))
            fig.savefig(output_file, dpi=100)

Step 1 load model A and corresponding data

In [12]:
models_path = '/home/maellef/Results/best_models/converted' 
dataset_path = '/home/maellef/DataBase/fMRI_Embeddings' 
stimuli_path = '/home/maellef/DataBase/stimuli'  
outpath = '/home/maellef/Results'
save_path = os.path.join(outpath, 'General_phantom')
os.makedirs(save_path, exist_ok=True) 

In [13]:
def load_sub_models(sub, scale, conv, models_path, no_init=False): 
    models = {}
    #scale_path = os.path.join(models_path, sub, scale)
    for model in os.listdir(models_path):
        if '.pt' in model and conv in model and sub in model and scale in model:
            model_path = os.path.join(models_path, model)
            modeldict = torch.load(model_path, map_location=torch.device('cpu'))
            model_net = encod.SoundNetEncoding_conv(out_size=modeldict['out_size'],output_layer=modeldict['output_layer'],
                                                    kernel_size=modeldict['kernel_size'], no_init=no_init)
            if not no_init:
                model_net.load_state_dict(modeldict['checkpoint'])
            models[model] = model_net
    return models

In [20]:
no_init = False
conv = 'conv4' #'opt110_wb'
sub = 'sub-01'#, 'sub-02', 'sub-03', 'sub-04', 'sub-05'
scale = 'MIST_ROI'#, 'auditory_Voxels' 

models = load_sub_models(sub, scale, conv, models_path, no_init=no_init)

dataset = 'friends'
sub_dataset = 's04'
data_stimuli_path = os.path.join(stimuli_path, dataset, sub_dataset)
parcellation_path = os.path.join(dataset_path, scale, dataset, sub)
pairs_wav_fmri = fu.associate_stimuli_with_Parcellation(data_stimuli_path, parcellation_path)    

shape of encoding matrice from last encoding layer : 1024 X 210


ste 2.1 get output of model conv 7
step 2.2 train ridge regression on Friends s4 with output of model

In [ ]:
def test(testloader,net, epoch, mseloss, gpu=True):
    all_fmri = []
    all_fmri_p = []
    net.eval()
    with torch.no_grad():
        for (wav,fmri) in testloader:
            bsize = wav.shape[0]
            
            # load data
            wav = torch.Tensor(wav).view(1,1,-1,1)
            if gpu:
                wav = wav.cuda()

            fmri = fmri.view(bsize,-1)
            if gpu:
                fmri=fmri.cuda()

            # Forward pass
            fmri_p = net(wav, epoch).permute(2,1,0,3).squeeze()

            #Cropping the end of the predicted fmri to match the measured bold
            fmri_p = fmri_p[:bsize]
            
            all_fmri.append(fmri.cpu().numpy().reshape(bsize,-1))
            all_fmri_p.append(fmri_p.cpu().numpy().reshape(bsize,-1))

    r2_model = r2_score(np.vstack(all_fmri),np.vstack(all_fmri_p),multioutput='raw_values')
    return r2_model

In [40]:
shape = 210 if scale == 'MIST_ROI' else 556
for name, model in models.items():
    pass
batchsize = int(name[name.find('conv_')+len('conv_'):name.find('conv_')+len('conv_')+3])

for pair in pairs_wav_fmri:
    eval_input = [pair] if len(pair) == 2 else [(pair[0], pair[1])]
    xTest, yTest = create_usable_audiofmri_datasets(eval_input, tr=1.49, sr=22050, name='test')
    TestDataset = SequentialDataset(xTest, yTest, batch_size=batchsize, selection=None)
    testloader = DataLoader(TestDataset, batch_size=None)

getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...


/home/maellef/git/cNeuromod_encoding_2020/Datasets_utils.py:99: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  length = librosa.get_duration(filename = audio_path)


getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
done.
getting audio files for test...
getting fMRI files for test...
don

In [ ]:
def one_run_eval(sub, dataset, models_dict, pairs_wav_fmri, no_init=False, outpath = save_path):
    shape = 210 if scale == 'MIST_ROI' else 556
    for name, model in models_dict.items():
        all_runs = np.array([]).reshape(-1,shape)
        batchsize = int(name[name.find('conv_')+len('conv_'):name.find('conv_')+len('conv_')+3])
        print('batchsize '+str(batchsize))
        for pair in pairs_wav_fmri:
            eval_input = [pair] if len(pair) == 2 else [(pair[0], pair[1])]
            xTest, yTest = create_usable_audiofmri_datasets(eval_input, tr=1.49, sr=22050, name='test')
            TestDataset = SequentialDataset(xTest, yTest, batch_size=batchsize, selection=None)
            testloader = DataLoader(TestDataset, batch_size=None)    
            r2_score = test_r2(testloader, net=model, epoch=1, mseloss=nn.MSELoss(reduction='sum'), gpu=False)
            all_runs = np.append(all_runs, r2_score.reshape(1,-1), axis=0)
        print(all_runs.shape)

        #sub_path = os.path.join(save_path, dataset, sub)
        #sub_path = sub_path if not no_init else os.path.join(sub_path, 'no_init_model')
        #os.makedirs(sub_path, exist_ok=True)
        sub_path = outpath
        savepath = os.path.join(sub_path, name[:-3])
        np.save(savepath, all_runs)

step 3 evaluate ridge regrssion result on a film from Movie 10

In [ ]:
dataset = 'friends' #, 'movie10'
sub_dataset = 's04' #, 'bourne', 'figures', 'life', 'wolf'
data_stimuli_path = os.path.join(stimuli_path, dataset, sub_dataset)

step 4 visualize predicted output by ridge regression

In [ ]:
result_path = os.path.join(save_path, dataset, sub_dataset)
os.makedirs(result_path, exist_ok=True) 

In [ ]:
alphas = np.logspace(0.1, 3, 10)
    group_kfold = GroupKFold(n_splits=n_splits)
    cv = group_kfold.split(X, y, groups)

    model = RidgeCV(
        alphas=alphas,
        fit_intercept=True,
        #normalize=False,
        cv=cv,
    )

    return model.fit(X, y)